In [14]:
#COMMENT1
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.misc import imshow
import cv2
import os
import time
from hashlib import md5
import scipy
import imageio

## Finding Similar images

In [15]:
IMAGE_DIR = 'C:\\Users\\Patronus\\Desktop\\ml-school\\dev_dataset\\'
os.chdir(IMAGE_DIR)
os.getcwd()

'C:\\Users\\Patronus\\Desktop\\ml-school\\dev_dataset'

In [16]:
image_files = os.listdir()


In [17]:
def filter_images(images):
    image_list = []
    for image in images:
        try:
            assert imageio.imread(image).shape[2] == 3
            image_list.append(image)
        except  AssertionError as e:
            print(e)
    return image_list

In [18]:
def img_gray(image):
    image = imageio.imread(image)
    return np.average(image, weights=[0.299, 0.587, 0.114], axis=2)

In [19]:
def resize(image, height=30, width=30):
    row_res = cv2.resize(image,(height, width), interpolation = cv2.INTER_AREA).flatten()
    col_res = cv2.resize(image,(height, width), interpolation = cv2.INTER_AREA).flatten('F')
    return row_res, col_res

In [20]:
def intensity_diff(row_res, col_res):
    difference_row = np.diff(row_res)
    difference_col = np.diff(col_res)
    difference_row = difference_row > 0
    difference_col = difference_col > 0
    return np.vstack((difference_row, difference_col)).flatten()

In [21]:
def difference_score(image, height = 30, width = 30):
    gray = img_gray(image)
    row_res, col_res = resize(gray, height, width)
    difference = intensity_diff(row_res, col_res)
    
    return difference

In [22]:
def hamming_distance(image, image2):
    score =scipy.spatial.distance.hamming(image, image2)
    return score

In [23]:
def difference_score_dict(image_list):
    ds_dict = {}
    duplicates = []
    for image in image_list:
        ds = difference_score(image)
        
        if image not in ds_dict:
            ds_dict[image] = ds
        else:
            duplicates.append((image, ds_dict[image]) )
    
    return  duplicates, ds_dict

In [24]:
image_files = filter_images(image_files)
duplicates, ds_dict =difference_score_dict(image_files)

In [25]:
import itertools
for k1,k2 in itertools.combinations(ds_dict, 2):
    if hamming_distance(ds_dict[k1], ds_dict[k2])< .40:
        duplicates.append((k1,k2))

In [26]:
duplicates


[('1.jpg', '1_duplicate.jpg'),
 ('11.jpg', '11_duplicate.jpg'),
 ('11.jpg', '11_modification.jpg'),
 ('11_duplicate.jpg', '11_modification.jpg'),
 ('15.jpg', '15_modification.jpg'),
 ('4.jpg', '4_similar.jpg'),
 ('6.jpg', '6_similar.jpg')]